In [1]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [2]:
#the file must be loaded from drive:
file_name="/content/drive/My Drive/Assignment_4"

#Loading important Libraries:

In [3]:
import numpy as np
import os
import cv2

In [4]:
from keras.models import Sequential
from keras.layers import Conv2D
from keras.layers import Flatten
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import MaxPooling2D
from keras.optimizers import Adam, SGD, RMSprop, Nadam
from keras import backend as k

Using TensorFlow backend.


In this model we are using different types of Optimizers.Lets understand optimizer in brief:
Def: Optimizers are algorithms or methods used to change the attributes of your neural network such as weights and learning rate in order to reduce the losses.
Various types are:
1. SGD
2. Nesterov Accelerated Gradient Descent
3. RMSprop
4. Adam
5. Nadam


Lets start with SGD ie Stochastic gradient descent.
Stochastic gradient descent is an iterative method for optimizing an objective function with suitable smoothness properties.

Momentum in SGD: with momentum SGD remembers the update at each iteration, and determines the next update as a linear combination of the gradient and the previous update.


In [6]:
# initializing the CNN:
cnn_classifier=Sequential()

# Step1:Adding 1st layer of Convulation:
cnn_classifier.add(Conv2D(32,(3,3), input_shape=(64,64,3),activation='relu'))

##Step2: Pooling:
cnn_classifier.add(MaxPooling2D(pool_size=(2,2)))

# Repeat ie Adding Second Layer of Convulation:
cnn_classifier.add(Conv2D(32,(3,3),activation='relu'))

# Repeat ie Pooling:
cnn_classifier.add(MaxPooling2D(pool_size=(2,2)))

#Step3: Flattening:
cnn_classifier.add(Flatten())

#Step4: Full Connection:
cnn_classifier.add(Dense(units=128,activation='relu'))
cnn_classifier.add(Dense(units=1,activation='sigmoid'))

#Step5:Compiling The CNN:
cnn_classifier.compile(optimizer=SGD(learning_rate=0.001, momentum=0.9),loss='binary_crossentropy',metrics=['accuracy'])

cnn_classifier.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_3 (Conv2D)            (None, 62, 62, 32)        896       
_________________________________________________________________
max_pooling2d_3 (MaxPooling2 (None, 31, 31, 32)        0         
_________________________________________________________________
conv2d_4 (Conv2D)            (None, 29, 29, 32)        9248      
_________________________________________________________________
max_pooling2d_4 (MaxPooling2 (None, 14, 14, 32)        0         
_________________________________________________________________
flatten_2 (Flatten)          (None, 6272)              0         
_________________________________________________________________
dense_3 (Dense)              (None, 128)               802944    
_________________________________________________________________
dense_4 (Dense)              (None, 1)                

In [11]:
from keras.preprocessing.image import ImageDataGenerator
train_datagen = ImageDataGenerator(rescale=1./255)


test_datagen = ImageDataGenerator(rescale=1./255)

training_set =  train_datagen.flow_from_directory('/content/drive/My Drive/Assignment_4/Train_data',
                                                  target_size=(64, 64),
                                                   batch_size=8,
                                                  class_mode='binary',
                                                  shuffle=True)
testing_set = test_datagen.flow_from_directory('/content/drive/My Drive/Assignment_4/test_data',
                                               target_size=(64, 64),
                                               batch_size=8,
                                               class_mode='binary')

Found 1274 images belonging to 2 classes.
Found 160 images belonging to 2 classes.


In this part we will learn about callbacks in keras with its implementation.

Def: A callback is a set of functions to be applied at given stages of the training procedure.
We define and use a callback when we want to automate some tasks after every training/epoch that help us to control the training process. This includes stopping training when you reach a certain accuracy/loss score, saving model as a checkpoint after each successful epoch.

EarlyStopping:
 One way to avoid overfitting is to terminate the process early. The EarlyStoppingfunction has various metrics/arguments that we can modify to set up when the training process should stop. 

Reduce learning rate:
Models often benefit from reducing the learning rate by a factor of 2-10 once learning stagnates. This callback monitors a quantity and if no improvement is seen for a 'patience' number of epochs, the learning rate is reduced

In [12]:
from keras import callbacks
call_backs=callbacks.ModelCheckpoint('model_callbacks.h5',
                                      monitor='val_loss',
                                      mode='min',
                                      save_best_only=True,
                                      verbose=1)

early_stopping= callbacks.EarlyStopping(monitor='val_loss',
                            min_delta=0,
                            patience=5,
                            verbose=1,
                            restore_best_weights=True)
red_on_plt=callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.002, patience=3, verbose=1, min_delta=0)

In [13]:
history1=cnn_classifier.fit_generator(training_set,
                             steps_per_epoch=1274//8,
                             epochs=20,
                             validation_data=testing_set,
                             validation_steps=250,
                             callbacks=[early_stopping,call_backs,red_on_plt])

Epoch 1/20
159/159 [==============================] - 616s 4s/step - loss: 0.6403 - accuracy: 0.6295 - val_loss: 0.7469 - val_accuracy: 0.3535

Epoch 00001: val_loss improved from inf to 0.74695, saving model to model_callbacks.h5
Epoch 2/20
159/159 [==============================] - 11s 71ms/step - loss: 0.5215 - accuracy: 0.7425 - val_loss: 0.8551 - val_accuracy: 0.3720

Epoch 00002: val_loss did not improve from 0.74695
Epoch 3/20
159/159 [==============================] - 11s 72ms/step - loss: 0.4730 - accuracy: 0.7694 - val_loss: 1.3758 - val_accuracy: 0.2920

Epoch 00003: val_loss did not improve from 0.74695
Epoch 4/20
159/159 [==============================] - 11s 71ms/step - loss: 0.4245 - accuracy: 0.8073 - val_loss: 0.8068 - val_accuracy: 0.2525

Epoch 00004: val_loss did not improve from 0.74695

Epoch 00004: ReduceLROnPlateau reducing learning rate to 2.0000000949949025e-06.
Epoch 5/20
159/159 [==============================] - 11s 72ms/step - loss: 0.4162 - accuracy: 0.78

In [14]:
# initializing the CNN:
cnn_classifier=Sequential()

# Step1:Adding 1st layer of Convulation:
cnn_classifier.add(Conv2D(32,(3,3), input_shape=(64,64,3),activation='relu'))

##Step2: Pooling:
cnn_classifier.add(MaxPooling2D(pool_size=(2,2)))

# Repeat ie Adding Second Layer of Convulation:
cnn_classifier.add(Conv2D(64,(3,3),activation='relu'))

# Repeat ie Pooling:
cnn_classifier.add(MaxPooling2D(pool_size=(2,2)))

# Adding 3rd layer:
cnn_classifier.add(Conv2D(128, kernel_size=(3,3), activation='relu'))
cnn_classifier.add(MaxPooling2D(pool_size=(2,2)))

#Step3: Flattening:
cnn_classifier.add(Flatten())

#Step4: Full Connection:
cnn_classifier.add(Dense(units=128,activation='relu'))
cnn_classifier.add(Dropout(0.10))
cnn_classifier.add(Dense(units=1,activation='sigmoid'))

#Step5:Compiling The CNN:
cnn_classifier.compile(optimizer=SGD(learning_rate=0.001, momentum=0.9),loss='binary_crossentropy',metrics=['accuracy'])

cnn_classifier.summary()

Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_5 (Conv2D)            (None, 62, 62, 32)        896       
_________________________________________________________________
max_pooling2d_5 (MaxPooling2 (None, 31, 31, 32)        0         
_________________________________________________________________
conv2d_6 (Conv2D)            (None, 29, 29, 64)        18496     
_________________________________________________________________
max_pooling2d_6 (MaxPooling2 (None, 14, 14, 64)        0         
_________________________________________________________________
conv2d_7 (Conv2D)            (None, 12, 12, 128)       73856     
_________________________________________________________________
max_pooling2d_7 (MaxPooling2 (None, 6, 6, 128)         0         
_________________________________________________________________
flatten_3 (Flatten)          (None, 4608)             

In [ ]:
from keras.preprocessing.image import ImageDataGenerator
train_datagen = ImageDataGenerator(rescale=1./255)


test_datagen = ImageDataGenerator(rescale=1./255)

training_set =  train_datagen.flow_from_directory('/content/drive/My Drive/Assignment_4/Train_data',
                                                  target_size=(64, 64),
                                                   batch_size=16,
                                                  class_mode='binary')
testing_set = test_datagen.flow_from_directory('/content/drive/My Drive/Assignment_4/test_data',
                                               target_size=(64, 64),
                                               batch_size=16,
                                               class_mode='binary')

Found 1274 images belonging to 2 classes.
Found 160 images belonging to 2 classes.


In [ ]:
from keras import callbacks
call_backs=callbacks.ModelCheckpoint('model_callbacks.h5',
                                      monitor='val_loss',
                                      mode='min',
                                      save_best_only=True,
                                      verbose=1)

early_stopping= callbacks.EarlyStopping(monitor='val_loss',
                            min_delta=0,
                            patience=5,
                            verbose=1,
                            restore_best_weights=True)
red_on_plt=callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.001, patience=3, verbose=1, min_delta=0)

In [15]:
history1=cnn_classifier.fit_generator(training_set,
                             steps_per_epoch=500,
                             epochs=20,
                             validation_data=testing_set,
                             validation_steps=500,
                             callbacks=[early_stopping,call_backs,red_on_plt])

Epoch 1/20
500/500 [==============================] - 29s 58ms/step - loss: 0.6399 - accuracy: 0.6384 - val_loss: 0.7114 - val_accuracy: 0.4000

Epoch 00001: val_loss did not improve from 0.60697
Epoch 2/20
500/500 [==============================] - 29s 57ms/step - loss: 0.4657 - accuracy: 0.7823 - val_loss: 1.3132 - val_accuracy: 0.2625

Epoch 00002: val_loss did not improve from 0.60697
Epoch 3/20
500/500 [==============================] - 29s 58ms/step - loss: 0.2966 - accuracy: 0.8732 - val_loss: 2.5225 - val_accuracy: 0.1562

Epoch 00003: val_loss did not improve from 0.60697
Epoch 4/20
500/500 [==============================] - 28s 57ms/step - loss: 0.2039 - accuracy: 0.9180 - val_loss: 3.9744 - val_accuracy: 0.1250

Epoch 00004: val_loss did not improve from 0.60697

Epoch 00004: ReduceLROnPlateau reducing learning rate to 2.0000000949949025e-06.
Epoch 5/20
500/500 [==============================] - 29s 57ms/step - loss: 0.2438 - accuracy: 0.8844 - val_loss: 4.3755 - val_accurac

In [16]:
cnn_classifier=Sequential()
cnn_classifier.add(Conv2D(32, kernel_size=(3,3),input_shape=(200,200,3), activation='relu'))
cnn_classifier.add(MaxPooling2D(2,2))
cnn_classifier.add(Conv2D(64, kernel_size=(3,3), activation='relu'))
cnn_classifier.add(MaxPooling2D(2,2))
cnn_classifier.add(Conv2D(128, kernel_size=(3,3), activation='relu'))
cnn_classifier.add(MaxPooling2D(2,2))
cnn_classifier.add(Dropout(0.3))

cnn_classifier.add(Flatten())

cnn_classifier.add(Dense(128, activation='relu'))
cnn_classifier.add(Dropout(0.25))
cnn_classifier.add(Dense(1, activation='softmax'))

cnn_classifier.compile(SGD(learning_rate=0.001, momentum=0.9), metrics=['accuracy'], loss='binary_crossentropy')

cnn_classifier.summary()

Model: "sequential_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_8 (Conv2D)            (None, 198, 198, 32)      896       
_________________________________________________________________
max_pooling2d_8 (MaxPooling2 (None, 99, 99, 32)        0         
_________________________________________________________________
conv2d_9 (Conv2D)            (None, 97, 97, 64)        18496     
_________________________________________________________________
max_pooling2d_9 (MaxPooling2 (None, 48, 48, 64)        0         
_________________________________________________________________
conv2d_10 (Conv2D)           (None, 46, 46, 128)       73856     
_________________________________________________________________
max_pooling2d_10 (MaxPooling (None, 23, 23, 128)       0         
_________________________________________________________________
dropout_2 (Dropout)          (None, 23, 23, 128)      

In [23]:
from keras.preprocessing.image import ImageDataGenerator
train_datagen = ImageDataGenerator(rescale=1./255)


test_datagen = ImageDataGenerator(rescale=1./255)

training_set =  train_datagen.flow_from_directory('/content/drive/My Drive/Assignment_4/Train_data',
                                                  target_size=(200, 200),
                                                   batch_size=16,
                                                  class_mode='binary')
testing_set = test_datagen.flow_from_directory('/content/drive/My Drive/Assignment_4/test_data',
                                               target_size=(200,200),
                                               batch_size=16,
                                               class_mode='binary')

Found 1274 images belonging to 2 classes.
Found 160 images belonging to 2 classes.


In [24]:
from keras import callbacks
call_backs=callbacks.ModelCheckpoint('model_callbacks.h5',
                                      monitor='val_loss',
                                      mode='min',
                                      save_best_only=True,
                                      verbose=1)

early_stopping= callbacks.EarlyStopping(monitor='val_loss',
                            min_delta=0,
                            patience=4,
                            verbose=1,
                            restore_best_weights=True)
red_on_plt=callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.001, patience=4, verbose=1, min_delta=0)

In [25]:
history1e=cnn_classifier.fit_generator(training_set,
                             steps_per_epoch=1274//16,
                             epochs=20,
                             validation_data=testing_set,
                             validation_steps=250,
                             callbacks=[early_stopping,call_backs,red_on_plt])

Epoch 1/20
79/79 [==============================] - 21s 260ms/step - loss: 7.7417 - accuracy: 0.4960 - val_loss: 10.4839 - val_accuracy: 0.5000

Epoch 00001: val_loss improved from inf to 10.48385, saving model to model_callbacks.h5
Epoch 2/20
79/79 [==============================] - 20s 258ms/step - loss: 7.7300 - accuracy: 0.4944 - val_loss: 9.5308 - val_accuracy: 0.5000

Epoch 00002: val_loss improved from 10.48385 to 9.53077, saving model to model_callbacks.h5
Epoch 3/20
79/79 [==============================] - 20s 256ms/step - loss: 7.6594 - accuracy: 0.5000 - val_loss: 5.7185 - val_accuracy: 0.5000

Epoch 00003: val_loss improved from 9.53077 to 5.71846, saving model to model_callbacks.h5
Epoch 4/20
79/79 [==============================] - 20s 257ms/step - loss: 7.6738 - accuracy: 0.5000 - val_loss: 7.6246 - val_accuracy: 0.5000

Epoch 00004: val_loss did not improve from 5.71846
Epoch 5/20
79/79 [==============================] - 20s 258ms/step - loss: 7.6981 - accuracy: 0.4984 

In [26]:
 ##    My accuracy is not improving.

Nesterov

In [31]:
# initializing the CNN:
cnn_classifier1=Sequential()

# Step1:Adding 1st layer of Convulation:
cnn_classifier1.add(Conv2D(32,(3,3), input_shape=(64,64,3),activation='relu'))

##Step2: Pooling:
cnn_classifier1.add(MaxPooling2D(pool_size=(2,2)))

# Repeat ie Adding Second Layer of Convulation:
cnn_classifier1.add(Conv2D(32,(3,3),activation='relu'))

# Repeat ie Pooling:
cnn_classifier1.add(MaxPooling2D(pool_size=(2,2)))

#Step3: Flattening:
cnn_classifier1.add(Flatten())

#Step4: Full Connection:
cnn_classifier1.add(Dense(units=128,activation='relu'))
cnn_classifier1.add(Dense(units=1,activation='sigmoid'))

#Step5:Compiling The CNN:
cnn_classifier1.compile(optimizer=SGD(learning_rate=0.001, momentum=0.9,nesterov='True'),loss='binary_crossentropy',metrics=['accuracy'])

cnn_classifier1.summary()

Model: "sequential_6"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_13 (Conv2D)           (None, 62, 62, 32)        896       
_________________________________________________________________
max_pooling2d_13 (MaxPooling (None, 31, 31, 32)        0         
_________________________________________________________________
conv2d_14 (Conv2D)           (None, 29, 29, 32)        9248      
_________________________________________________________________
max_pooling2d_14 (MaxPooling (None, 14, 14, 32)        0         
_________________________________________________________________
flatten_6 (Flatten)          (None, 6272)              0         
_________________________________________________________________
dense_11 (Dense)             (None, 128)               802944    
_________________________________________________________________
dense_12 (Dense)             (None, 1)                

In [32]:
from keras.preprocessing.image import ImageDataGenerator
train_datagen = ImageDataGenerator(rescale=1./255)


test_datagen = ImageDataGenerator(rescale=1./255)

training_set =  train_datagen.flow_from_directory('/content/drive/My Drive/Assignment_4/Train_data',
                                                  target_size=(64, 64),
                                                   batch_size=16,
                                                  class_mode='binary',
                                                  shuffle=True)
testing_set = test_datagen.flow_from_directory('/content/drive/My Drive/Assignment_4/test_data',
                                               target_size=(64, 64),
                                               batch_size=16,
                                               class_mode='binary')

Found 1274 images belonging to 2 classes.
Found 160 images belonging to 2 classes.


In [33]:
from keras import callbacks
call_backs=callbacks.ModelCheckpoint('model1_callbacks.h5',
                                      monitor='val_loss',
                                      mode='min',
                                      save_best_only=True,
                                      verbose=1)

early_stopping= callbacks.EarlyStopping(monitor='val_loss',
                            min_delta=0,
                            patience=5,
                            verbose=1,
                            restore_best_weights=True)
red_on_plt=callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.001, patience=3, verbose=1, min_delta=0)

In [34]:
history1=cnn_classifier1.fit_generator(training_set,
                             steps_per_epoch=1274//16,
                             epochs=20,
                             validation_data=testing_set,
                             validation_steps=250,
                             callbacks=[early_stopping,call_backs,red_on_plt])

Epoch 1/20
79/79 [==============================] - 18s 223ms/step - loss: 0.6718 - accuracy: 0.5866 - val_loss: 0.7119 - val_accuracy: 0.4125

Epoch 00001: val_loss improved from inf to 0.71190, saving model to model1_callbacks.h5
Epoch 2/20
79/79 [==============================] - 18s 229ms/step - loss: 0.6115 - accuracy: 0.6820 - val_loss: 0.9360 - val_accuracy: 0.3562

Epoch 00002: val_loss did not improve from 0.71190
Epoch 3/20
79/79 [==============================] - 17s 216ms/step - loss: 0.5328 - accuracy: 0.7496 - val_loss: 1.1517 - val_accuracy: 0.2875

Epoch 00003: val_loss did not improve from 0.71190
Epoch 4/20
79/79 [==============================] - 18s 234ms/step - loss: 0.4672 - accuracy: 0.7774 - val_loss: 1.2786 - val_accuracy: 0.2812

Epoch 00004: val_loss did not improve from 0.71190

Epoch 00004: ReduceLROnPlateau reducing learning rate to 1.0000000474974512e-06.
Epoch 5/20
79/79 [==============================] - 18s 222ms/step - loss: 0.4903 - accuracy: 0.7464 

RMSprop

In [36]:
# initializing the CNN:
cnn_classifier2=Sequential()

# Step1:Adding 1st layer of Convulation:
cnn_classifier2.add(Conv2D(32,(3,3), input_shape=(64,64,3),activation='relu'))

##Step2: Pooling:
cnn_classifier2.add(MaxPooling2D(pool_size=(2,2)))

# Repeat ie Adding Second Layer of Convulation:
cnn_classifier2.add(Conv2D(64,(3,3),activation='relu'))

# Repeat ie Pooling:
cnn_classifier2.add(MaxPooling2D(pool_size=(2,2)))

# Adding 3rd layer:
cnn_classifier2.add(Conv2D(128, kernel_size=(3,3), activation='relu'))
cnn_classifier2.add(MaxPooling2D(pool_size=(2,2)))

#Step3: Flattening:
cnn_classifier2.add(Flatten())

#Step4: Full Connection:
cnn_classifier2.add(Dense(units=128,activation='relu'))
cnn_classifier2.add(Dropout(0.50))
cnn_classifier2.add(Dense(units=1,activation='sigmoid'))

#Step5:Compiling The CNN:
cnn_classifier2.compile(RMSprop(), metrics=['accuracy'], loss='binary_crossentropy')

cnn_classifier2.summary()

Model: "sequential_8"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_18 (Conv2D)           (None, 62, 62, 32)        896       
_________________________________________________________________
max_pooling2d_18 (MaxPooling (None, 31, 31, 32)        0         
_________________________________________________________________
conv2d_19 (Conv2D)           (None, 29, 29, 64)        18496     
_________________________________________________________________
max_pooling2d_19 (MaxPooling (None, 14, 14, 64)        0         
_________________________________________________________________
conv2d_20 (Conv2D)           (None, 12, 12, 128)       73856     
_________________________________________________________________
max_pooling2d_20 (MaxPooling (None, 6, 6, 128)         0         
_________________________________________________________________
flatten_8 (Flatten)          (None, 4608)             

In [38]:
from keras.preprocessing.image import ImageDataGenerator
train_datagen = ImageDataGenerator(rescale=1./255)


test_datagen = ImageDataGenerator(rescale=1./255)

training_set =  train_datagen.flow_from_directory('/content/drive/My Drive/Assignment_4/Train_data',
                                                  target_size=(64, 64),
                                                   batch_size=16,
                                                  class_mode='binary',
                                                  shuffle=True)
testing_set = test_datagen.flow_from_directory('/content/drive/My Drive/Assignment_4/test_data',
                                               target_size=(64, 64),
                                               batch_size=16,
                                               class_mode='binary')

Found 1274 images belonging to 2 classes.
Found 160 images belonging to 2 classes.


In [39]:
from keras import callbacks
call_backs=callbacks.ModelCheckpoint('model2_callbacks.h5',
                                      monitor='val_loss',
                                      mode='min',
                                      save_best_only=True,
                                      verbose=1)

early_stopping= callbacks.EarlyStopping(monitor='val_loss',
                            min_delta=0,
                            patience=8,
                            verbose=1,
                            restore_best_weights=True)
red_on_plt=callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.001, patience=5, verbose=1, min_delta=0)

In [40]:
history1=cnn_classifier1.fit_generator(training_set,
                             steps_per_epoch=1274//16,
                             epochs=20,
                             validation_data=testing_set,
                             validation_steps=1000,
                             callbacks=[early_stopping,call_backs,red_on_plt])

Epoch 1/20
79/79 [==============================] - 58s 737ms/step - loss: 0.6415 - accuracy: 0.6113 - val_loss: 0.7816 - val_accuracy: 0.4125

Epoch 00001: val_loss improved from inf to 0.78158, saving model to model2_callbacks.h5
Epoch 2/20
79/79 [==============================] - 56s 712ms/step - loss: 0.6407 - accuracy: 0.6179 - val_loss: 0.7301 - val_accuracy: 0.4125

Epoch 00002: val_loss improved from 0.78158 to 0.73014, saving model to model2_callbacks.h5
Epoch 3/20
79/79 [==============================] - 57s 720ms/step - loss: 0.6421 - accuracy: 0.6142 - val_loss: 0.7287 - val_accuracy: 0.4062

Epoch 00003: val_loss improved from 0.73014 to 0.72872, saving model to model2_callbacks.h5
Epoch 4/20
79/79 [==============================] - 57s 720ms/step - loss: 0.6388 - accuracy: 0.6256 - val_loss: 0.7201 - val_accuracy: 0.4125

Epoch 00004: val_loss improved from 0.72872 to 0.72012, saving model to model2_callbacks.h5
Epoch 5/20
79/79 [==============================] - 58s 736m

Adam

In [45]:
# initializing the CNN:
cnn_classifier2=Sequential()

# Step1:Adding 1st layer of Convulation:
cnn_classifier2.add(Conv2D(32,(3,3), input_shape=(64,64,3),activation='relu'))

##Step2: Pooling:
cnn_classifier2.add(MaxPooling2D(pool_size=(2,2)))

# Repeat ie Adding Second Layer of Convulation:
cnn_classifier2.add(Conv2D(64,(3,3),activation='relu'))

# Repeat ie Pooling:
cnn_classifier2.add(MaxPooling2D(pool_size=(2,2)))

# Again Repeating ie Adding Third Layer of Convulation:
cnn_classifier2.add(Conv2D(128,(3,3),activation='relu'))

# Same with the this ie Pooling:
cnn_classifier2.add(MaxPooling2D(pool_size=(2,2)))

#Step3: Flattening:
cnn_classifier2.add(Flatten())

#Step4: Full Connection:
cnn_classifier2.add(Dense(units=128,activation='relu'))
cnn_classifier2.add(Dense(units=1,activation='sigmoid'))

#Step5:Compiling The CNN:
cnn_classifier2.compile(optimizer='adam',loss='binary_crossentropy',metrics=['accuracy'])


In [46]:
from keras.preprocessing.image import ImageDataGenerator
train_datagen = ImageDataGenerator(rescale=1./255)


test_datagen = ImageDataGenerator(rescale=1./255)

training_set =  train_datagen.flow_from_directory('/content/drive/My Drive/Assignment_4/Train_data',
                                                  target_size=(64, 64),
                                                   batch_size=16,
                                                  class_mode='binary',
                                                  shuffle=True)
testing_set = test_datagen.flow_from_directory('/content/drive/My Drive/Assignment_4/test_data',
                                               target_size=(64, 64),
                                               batch_size=16,
                                               class_mode='binary')

Found 1274 images belonging to 2 classes.
Found 160 images belonging to 2 classes.


In [47]:
from keras import callbacks
call_backs1=callbacks.ModelCheckpoint('model2_callbacks.h5',
                                      monitor='val_loss',
                                      mode='min',
                                      save_best_only=True,
                                      verbose=1)

early_stopping= callbacks.EarlyStopping(monitor='val_loss',
                            min_delta=0,
                            patience=8,
                            verbose=1,
                            restore_best_weights=True)
red_on_plt=callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.001, patience=5, verbose=1, min_delta=0)

In [48]:
history1=cnn_classifier2.fit_generator(training_set,
                             steps_per_epoch=1274//16,
                             epochs=20,
                             validation_data=testing_set,
                             validation_steps=1000,
                             callbacks=[early_stopping,call_backs1,red_on_plt])

Epoch 1/20
79/79 [==============================] - 58s 733ms/step - loss: 0.5877 - accuracy: 0.6614 - val_loss: 0.9591 - val_accuracy: 0.2875

Epoch 00001: val_loss improved from inf to 0.95909, saving model to model2_callbacks.h5
Epoch 2/20
79/79 [==============================] - 57s 721ms/step - loss: 0.3401 - accuracy: 0.8362 - val_loss: 4.0771 - val_accuracy: 0.2500

Epoch 00002: val_loss did not improve from 0.95909
Epoch 3/20
79/79 [==============================] - 57s 722ms/step - loss: 0.2210 - accuracy: 0.9118 - val_loss: 4.1858 - val_accuracy: 0.1312

Epoch 00003: val_loss did not improve from 0.95909
Epoch 4/20
79/79 [==============================] - 56s 710ms/step - loss: 0.1133 - accuracy: 0.9634 - val_loss: 5.8719 - val_accuracy: 0.2313

Epoch 00004: val_loss did not improve from 0.95909
Epoch 5/20
79/79 [==============================] - 56s 705ms/step - loss: 0.1144 - accuracy: 0.9587 - val_loss: 8.5864 - val_accuracy: 0.1250

Epoch 00005: val_loss did not improve f

## Nadam

In [49]:
# initializing the CNN:
cnn_classifier3=Sequential()

# Step1:Adding 1st layer of Convulation:
cnn_classifier3.add(Conv2D(32,(3,3), input_shape=(64,64,3),activation='relu'))

##Step2: Pooling:
cnn_classifier3.add(MaxPooling2D(pool_size=(2,2)))

# Repeat ie Adding Second Layer of Convulation:
cnn_classifier3.add(Conv2D(64,(3,3),activation='relu'))

# Repeat ie Pooling:
cnn_classifier3.add(MaxPooling2D(pool_size=(2,2)))

# Again Repeating ie Adding Third Layer of Convulation:
cnn_classifier3.add(Conv2D(128,(3,3),activation='relu'))

# Same with the this ie Pooling:
cnn_classifier3.add(MaxPooling2D(pool_size=(2,2)))

#Step3: Flattening:
cnn_classifier3.add(Flatten())

#Step4: Full Connection:
cnn_classifier3.add(Dense(units=128,activation='relu'))
cnn_classifier3.add(Dense(units=1,activation='sigmoid'))

#Step5:Compiling The CNN:
cnn_classifier3.compile(optimizer='Nadam',loss='binary_crossentropy',metrics=['accuracy'])


In [50]:
from keras.preprocessing.image import ImageDataGenerator
train_datagen = ImageDataGenerator(rescale=1./255)


test_datagen = ImageDataGenerator(rescale=1./255)

training_set =  train_datagen.flow_from_directory('/content/drive/My Drive/Assignment_4/Train_data',
                                                  target_size=(64, 64),
                                                   batch_size=32,
                                                  class_mode='binary',
                                                  shuffle=True)
testing_set = test_datagen.flow_from_directory('/content/drive/My Drive/Assignment_4/test_data',
                                               target_size=(64, 64),
                                               batch_size=32,
                                               class_mode='binary')

Found 1274 images belonging to 2 classes.
Found 160 images belonging to 2 classes.


In [51]:
from keras import callbacks
call_backs2=callbacks.ModelCheckpoint('model2_callbacks.h5',
                                      monitor='val_loss',
                                      mode='min',
                                      save_best_only=True,
                                      verbose=1)

early_stopping= callbacks.EarlyStopping(monitor='val_loss',
                            min_delta=0,
                            patience=8,
                            verbose=1,
                            restore_best_weights=True)
red_on_plt=callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.001, patience=5, verbose=1, min_delta=0)

In [52]:
history1=cnn_classifier3.fit_generator(training_set,
                             steps_per_epoch=1274//32,
                             epochs=20,
                             validation_data=testing_set,
                             validation_steps=200,
                             callbacks=[early_stopping,call_backs2,red_on_plt])

Epoch 1/20
39/39 [==============================] - 24s 627ms/step - loss: 0.6519 - accuracy: 0.6208 - val_loss: 0.9635 - val_accuracy: 0.4062

Epoch 00001: val_loss improved from inf to 0.96346, saving model to model2_callbacks.h5
Epoch 2/20
39/39 [==============================] - 24s 619ms/step - loss: 0.4449 - accuracy: 0.7681 - val_loss: 1.3985 - val_accuracy: 0.2625

Epoch 00002: val_loss did not improve from 0.96346
Epoch 3/20
39/39 [==============================] - 24s 620ms/step - loss: 0.3427 - accuracy: 0.8406 - val_loss: 2.8856 - val_accuracy: 0.2188

Epoch 00003: val_loss did not improve from 0.96346
Epoch 4/20
39/39 [==============================] - 26s 660ms/step - loss: 0.2202 - accuracy: 0.9147 - val_loss: 4.2963 - val_accuracy: 0.1875

Epoch 00004: val_loss did not improve from 0.96346
Epoch 5/20
39/39 [==============================] - 24s 620ms/step - loss: 0.1383 - accuracy: 0.9501 - val_loss: 5.9745 - val_accuracy: 0.1125

Epoch 00005: val_loss did not improve f